<a href="https://colab.research.google.com/github/Arjavjain100/Apple-Scab-Detection/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torchvision import datasets, transforms, models
from tqdm.auto import tqdm
from torchinfo import summary
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

In [4]:
# Alexnet from scratch
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding=0))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, 1000))
        self.fc3= nn.Sequential(
            nn.Linear(1000, num_classes))

        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = torch.flatten(out, start_dim =1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [5]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [6]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)
    
            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [7]:

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [8]:
def plot_loss_curves(results):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [9]:
batch_size = 32

transform = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((227, 227)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((227, 227)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
}
test_data = datasets.ImageFolder('/content/drive/MyDrive/Minor Project 1/Augmented Dataset/Test', transform = transform['test'])
train_data = datasets.ImageFolder('/content/drive/MyDrive/Minor Project 1/Augmented Dataset/Train',  transform = transform['train'])
train_loader = torch.utils.data.DataLoader(
    train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(
    test_data, shuffle=True, batch_size=batch_size)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Applying transfer learning
alex = models.alexnet(pretrained=True)
net_add =  nn.Linear(1000, 2)

alex_pt = nn.Sequential(alex, net_add).to(device)
alex_scratch = AlexNet(2).to(device)

criterion = nn.CrossEntropyLoss()

optimizer_scratch = opt.Adam(alex_scratch.parameters(), lr =0.001)
optimizer_pt = opt.Adam(alex_pt.parameters(), lr =0.001)

In [12]:
summary(alex_scratch,input_size=[32, 3, 227, 227])

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [32, 2]                   --
├─Sequential: 1-1                        [32, 96, 27, 27]          --
│    └─Conv2d: 2-1                       [32, 96, 55, 55]          34,944
│    └─BatchNorm2d: 2-2                  [32, 96, 55, 55]          192
│    └─ReLU: 2-3                         [32, 96, 55, 55]          --
│    └─MaxPool2d: 2-4                    [32, 96, 27, 27]          --
├─Sequential: 1-2                        [32, 256, 13, 13]         --
│    └─Conv2d: 2-5                       [32, 256, 27, 27]         614,656
│    └─BatchNorm2d: 2-6                  [32, 256, 27, 27]         512
│    └─ReLU: 2-7                         [32, 256, 27, 27]         --
│    └─MaxPool2d: 2-8                    [32, 256, 13, 13]         --
├─Sequential: 1-3                        [32, 384, 13, 13]         --
│    └─Conv2d: 2-9                       [32, 384, 13, 13]         885,120

In [13]:
summary(alex_pt,input_size=[32, 3, 227, 227])

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 2]                   --
├─AlexNet: 1-1                           [32, 1000]                --
│    └─Sequential: 2-1                   [32, 256, 6, 6]           --
│    │    └─Conv2d: 3-1                  [32, 64, 56, 56]          23,296
│    │    └─ReLU: 3-2                    [32, 64, 56, 56]          --
│    │    └─MaxPool2d: 3-3               [32, 64, 27, 27]          --
│    │    └─Conv2d: 3-4                  [32, 192, 27, 27]         307,392
│    │    └─ReLU: 3-5                    [32, 192, 27, 27]         --
│    │    └─MaxPool2d: 3-6               [32, 192, 13, 13]         --
│    │    └─Conv2d: 3-7                  [32, 384, 13, 13]         663,936
│    │    └─ReLU: 3-8                    [32, 384, 13, 13]         --
│    │    └─Conv2d: 3-9                  [32, 256, 13, 13]         884,992
│    │    └─ReLU: 3-10                   [32, 256, 13, 13]        

In [ ]:
NUM_EPOCHS = 20

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

scratch_results = train(model=alex_scratch, 
                        train_dataloader=train_loader,
                        test_dataloader=test_loader,
                        optimizer=optimizer_scratch,
                        loss_fn=criterion, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 9.4666 | train_acc: 0.5359 | test_loss: 0.6461 | test_acc: 0.5469
Epoch: 2 | train_loss: 0.6605 | train_acc: 0.5898 | test_loss: 0.6532 | test_acc: 0.5469
Epoch: 3 | train_loss: 0.6391 | train_acc: 0.6014 | test_loss: 0.5939 | test_acc: 0.7891
Epoch: 4 | train_loss: 0.6315 | train_acc: 0.6241 | test_loss: 0.6668 | test_acc: 0.3516
Epoch: 5 | train_loss: 0.6121 | train_acc: 0.6445 | test_loss: 0.5460 | test_acc: 0.6484


In [ ]:
NUM_EPOCHS = 20

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

pt_results = train(model=alex_pt, 
                        train_dataloader=train_loader,
                        test_dataloader=test_loader,
                        optimizer=optimizer_pt,
                        loss_fn=criterion, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


In [ ]:
plot_loss_curves(scratch_results)

In [ ]:
plot_loss_curves(pt_results)